# Hate speech classification by k-fold cross validation on movies dataset



The class labels depict the following:

0: Normal speech, 
1: Offensive speech
2: Hate speech

#### To work with this, the following folder paths needs to be created in the directory of this notebook:
classification_reports/   : This will contain all the classification reports generated by the model

movies/       : contains all_movies.csv file

movies/for_training/:    contains 6 movies used for cross validation training and testing


In [1]:
! pip install transformers==2.6.0

     |████████████████████████████████| 2.6 MB 9.8 MB/s 
     |████████████████████████████████| 636 kB 59.4 MB/s 
     |████████████████████████████████| 895 kB 65.1 MB/s 
     |████████████████████████████████| 3.3 MB 57.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import re
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import os
import glob

In [3]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")


---
### Cross validation


#### 6-fold cross validation on movies 

Methods to convert the data into the data required by the model for training and testing

In [10]:
def convert_data_to_examples_cv(train, DATA_COLUMN, LABEL_COLUMN):
    train_InputExamples = train.apply(
        lambda x: InputExample(guid=None,  # Globally unique ID for bookkeeping, unused in this case
                               text_a=x[DATA_COLUMN],
                               text_b=None,
                               label=x[LABEL_COLUMN]), axis=1)

    return train_InputExamples


def convert_examples_to_tf_dataset_cv(examples, tokenizer, max_length=128):
    features = []  # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length,  # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True,  # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
                                                     input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )

In [11]:
def train_bert(df_train, df_test):
    # initialize model with 3 labels, for hate, offensive and normal class classification
    model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                            trainable=True,
                                                            num_labels=3)
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

    train = df_train[['text', 'majority_answer']]
    train.columns = ['DATA_COLUMN', 'LABEL_COLUMN']

    test = df_test[['text', 'majority_answer']]
    test.columns = ['DATA_COLUMN', 'LABEL_COLUMN']

    DATA_COLUMN = 'DATA_COLUMN'
    LABEL_COLUMN = 'LABEL_COLUMN'

    train_InputExamples = convert_data_to_examples_cv(train, DATA_COLUMN, LABEL_COLUMN)
    test_InputExamples = convert_data_to_examples_cv(test, DATA_COLUMN, LABEL_COLUMN)

    train_data = convert_examples_to_tf_dataset_cv(list(train_InputExamples), tokenizer)
    train_data = train_data.batch(32)

    valid_data = convert_examples_to_tf_dataset_cv(list(test_InputExamples), tokenizer)
    valid_data = valid_data.batch(32)

    # compile and fit
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-6, epsilon=1e-08, clipnorm=1.0),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])
    print('train data type',type(train_data))
    model.fit(train_data, epochs=6, validation_data=valid_data)

    test_data = convert_examples_to_tf_dataset_cv(list(test_InputExamples), tokenizer)
    test_data = test_data.batch(32)

    print('predicting')
    preds = model.predict(test_data)

    # classification
    return classification_report(pd.DataFrame(test['LABEL_COLUMN']), np.argmax(preds[0], axis=1), output_dict=True)

In [6]:
def load_movies_to_df(path):
    df_movies = []

    for filename in glob.glob(path + '*.csv'):
        df_movies.append(pd.read_csv(filename))

    return df_movies

In [7]:
df_movies = load_movies_to_df('movies/for_training/')
classification_reports = []
df_main = pd.DataFrame()

In [12]:
# perform cross folding
for i in range(len(df_movies)):
    df_train = pd.concat(df_movies[0:i] + df_movies[i + 1:])
    df_test = df_movies[i]

    train_movies = df_train['movie_name'].unique()
    test_movie = df_test['movie_name'].unique()
    print(','.join(train_movies))
    print(test_movie[0])

    report = train_bert(df_train, df_test)
    classification_reports.append(report)
    
    print('Train movies: ', str(','.join(train_movies)))
    print('Test movie: ', str(test_movie[0]))
    print('Classification report: \n', classification_reports[i])
    print('------------------------------------------------')

    df_cr = pd.DataFrame(classification_reports[i]).transpose()
    df_cr['movie_train'] =  str(','.join(train_movies))
    df_cr['movie_test'] = str(test_movie[0])
    df_cr.to_csv('classification_reports/'+'bert_cv_testmovie_'+str(test_movie[0])+'.csv')
    df_main = df_main.append(df_cr)

Pulp_Fiction,AmerricanHistoryX,TheWolfofWallStreet,Django_Unchained,South_Park
BlacKkKlansman


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2190: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


train data type <class 'tensorflow.python.data.ops.dataset_ops.BatchDataset'>
Epoch 1/6
    283/Unknown - 236s 767ms/step - loss: 0.6180 - accuracy: 0.8040WARNING:tensorflow:The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
283/283 [==============================] - 253s 827ms/step - loss: 0.6180 - accuracy: 0.8040 - val_loss: 0.3662 - val_accuracy: 0.8979
Epoch 2/6
283/283 [==============================] - 239s 843ms/step - loss: 0.2421 - accuracy: 0.9328 - val_loss: 0.2421 - val_accuracy: 0.9337
Epoch 3/6
283/283 [==============================] - 239s 843ms/step - loss: 0.1516 - accuracy: 0.9592 - val_loss: 0.2204 - val_accuracy: 0.9374
Epoch 4/6
283/283 [==============================] - 239s 843ms/step - loss: 0.1208 - accuracy: 0.9674 - val_loss: 0.2143 - val_accuracy: 0.9386
Epoch 5/6
283

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


train data type <class 'tensorflow.python.data.ops.dataset_ops.BatchDataset'>
Epoch 1/6
    284/Unknown - 242s 788ms/step - loss: 0.5010 - accuracy: 0.8516WARNING:tensorflow:The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
284/284 [==============================] - 259s 846ms/step - loss: 0.5010 - accuracy: 0.8516 - val_loss: 0.2814 - val_accuracy: 0.9334
Epoch 2/6
284/284 [==============================] - 239s 842ms/step - loss: 0.2091 - accuracy: 0.9336 - val_loss: 0.1857 - val_accuracy: 0.9396
Epoch 3/6
284/284 [==============================] - 239s 841ms/step - loss: 0.1485 - accuracy: 0.9558 - val_loss: 0.1647 - val_accuracy: 0.9482
Epoch 4/6
284/284 [==============================] - 239s 842ms/step - loss: 0.1171 - accuracy: 0.9646 - val_loss: 0.1644 - val_accuracy: 0.9470
Epoch 5/6
284

...

train data type <class 'tensorflow.python.data.ops.dataset_ops.BatchDataset'>
Epoch 1/6
    239/Unknown - 206s 786ms/step - loss: 0.5514 - accuracy: 0.7975WARNING:tensorflow:The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
239/239 [==============================] - 235s 908ms/step - loss: 0.5514 - accuracy: 0.7975 - val_loss: 0.2976 - val_accuracy: 0.8077
Epoch 2/6
239/239 [==============================] - 216s 904ms/step - loss: 0.2562 - accuracy: 0.9005 - val_loss: 0.1250 - val_accuracy: 0.9745
Epoch 3/6
239/239 [==============================] - 216s 905ms/step - loss: 0.1730 - accuracy: 0.9500 - val_loss: 0.0969 - val_accuracy: 0.9739
Epoch 4/6
239/239 [==============================] - 216s 905ms/step - loss: 0.1400 - accuracy: 0.9572 - val_loss: 0.0931 - val_accuracy: 0.9729
Epoch 5/6
239

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


train data type <class 'tensorflow.python.data.ops.dataset_ops.BatchDataset'>
Epoch 1/6
    280/Unknown - 239s 791ms/step - loss: 0.5958 - accuracy: 0.8244WARNING:tensorflow:The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
280/280 [==============================] - 257s 856ms/step - loss: 0.5958 - accuracy: 0.8244 - val_loss: 0.3800 - val_accuracy: 0.8941
Epoch 2/6
280/280 [==============================] - 237s 847ms/step - loss: 0.2050 - accuracy: 0.9399 - val_loss: 0.1837 - val_accuracy: 0.9033
Epoch 3/6
280/280 [==============================] - 242s 866ms/step - loss: 0.1516 - accuracy: 0.9540 - val_loss: 0.1231 - val_accuracy: 0.9708
Epoch 4/6
280/280 [==============================] - 237s 847ms/step - loss: 0.1238 - accuracy: 0.9623 - val_loss: 0.1066 - val_accuracy: 0.9720
Epoch 5/6
280

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


train data type <class 'tensorflow.python.data.ops.dataset_ops.BatchDataset'>
Epoch 1/6
    302/Unknown - 258s 792ms/step - loss: 0.5294 - accuracy: 0.8458WARNING:tensorflow:The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
302/302 [==============================] - 270s 831ms/step - loss: 0.5294 - accuracy: 0.8458 - val_loss: 0.2766 - val_accuracy: 0.9178
Epoch 2/6
302/302 [==============================] - 248s 823ms/step - loss: 0.1879 - accuracy: 0.9378 - val_loss: 0.2238 - val_accuracy: 0.9331
Epoch 3/6
302/302 [==============================] - 248s 822ms/step - loss: 0.1391 - accuracy: 0.9606 - val_loss: 0.2204 - val_accuracy: 0.9312
Epoch 4/6
302/302 [==============================] - 248s 822ms/step - loss: 0.1128 - accuracy: 0.9658 - val_loss: 0.2216 - val_accuracy: 0.9340
Epoch 5/6
302

In [13]:
df_main.to_csv('classification_reports/bert_crossvalid_movies.csv')

In [14]:
 print(df_main)

              precision  ...           movie_test
0              0.963112  ...       BlacKkKlansman
1              0.638298  ...       BlacKkKlansman
2              0.850000  ...       BlacKkKlansman
accuracy       0.940426  ...       BlacKkKlansman
macro avg      0.817137  ...       BlacKkKlansman
weighted avg   0.938045  ...       BlacKkKlansman
0              0.970610  ...         Pulp_Fiction
1              0.830258  ...         Pulp_Fiction
2              0.833333  ...         Pulp_Fiction
accuracy       0.945129  ...         Pulp_Fiction
macro avg      0.878067  ...         Pulp_Fiction
weighted avg   0.945649  ...         Pulp_Fiction
0              0.956781  ...    AmerricanHistoryX
1              0.817708  ...    AmerricanHistoryX
2              1.000000  ...    AmerricanHistoryX
accuracy       0.940575  ...    AmerricanHistoryX
macro avg      0.924830  ...    AmerricanHistoryX
weighted avg   0.939877  ...    AmerricanHistoryX
0              0.978732  ...  TheWolfofWallStreet


In [15]:
len(classification_reports[0])

6

In [16]:
df_main.head()

,precision,recall,f1-score,support,movie_train,movie_test
0,0.963112,0.982216,0.972570,1462.000000,"Pulp_Fiction,AmerricanHistoryX,TheWolfofWallSt...",BlacKkKlansman
1,0.638298,0.618557,0.628272,97.000000,"Pulp_Fiction,AmerricanHistoryX,TheWolfofWallSt...",BlacKkKlansman
2,0.850000,0.593023,0.698630,86.000000,"Pulp_Fiction,AmerricanHistoryX,TheWolfofWallSt...",BlacKkKlansman
accuracy,0.940426,0.940426,0.940426,0.940426,"Pulp_Fiction,AmerricanHistoryX,TheWolfofWallSt...",BlacKkKlansman
macro avg,0.817137,0.731265,0.766491,1645.000000,"Pulp_Fiction,AmerricanHistoryX,TheWolfofWallSt...",BlacKkKlansman


In [17]:
def get_precision_recall_f1(category, result_df):
    precision = result_df[result_df.label==category].precision.mean()
    recall = result_df[result_df.label==category].recall.mean()
    f1 = result_df[result_df.label==category]['f1-score'].mean()
    
    return {'label': category, 'precision': precision, 'recall': recall, 'f1': f1}

In [18]:
df_cv= pd.read_csv('classification_reports/bert_crossvalid_movies.csv')

In [19]:
len(classification_reports[0])

6

In [20]:
df_main.head()

,precision,recall,f1-score,support,movie_train,movie_test
0,0.963112,0.982216,0.972570,1462.000000,"Pulp_Fiction,AmerricanHistoryX,TheWolfofWallSt...",BlacKkKlansman
1,0.638298,0.618557,0.628272,97.000000,"Pulp_Fiction,AmerricanHistoryX,TheWolfofWallSt...",BlacKkKlansman
2,0.850000,0.593023,0.698630,86.000000,"Pulp_Fiction,AmerricanHistoryX,TheWolfofWallSt...",BlacKkKlansman
accuracy,0.940426,0.940426,0.940426,0.940426,"Pulp_Fiction,AmerricanHistoryX,TheWolfofWallSt...",BlacKkKlansman
macro avg,0.817137,0.731265,0.766491,1645.000000,"Pulp_Fiction,AmerricanHistoryX,TheWolfofWallSt...",BlacKkKlansman


In [21]:
def get_precision_recall_f1(category, result_df):
    precision = result_df[result_df.label==category].precision.mean()
    recall = result_df[result_df.label==category].recall.mean()
    f1 = result_df[result_df.label==category]['f1-score'].mean()
    
    return {'label': category, 'precision': precision, 'recall': recall, 'f1': f1}

In [25]:
df_cv= pd.read_csv('classification_reports/bert_crossvalid_movies.csv')

In [26]:
df_cv = df_cv.rename(columns={'Unnamed: 0': 'label', 'b': 'Y'})
df_cv.head()

,label,precision,recall,f1-score,support,movie_train,movie_test
0,0,0.963112,0.982216,0.972570,1462.000000,"Pulp_Fiction,AmerricanHistoryX,TheWolfofWallSt...",BlacKkKlansman
1,1,0.638298,0.618557,0.628272,97.000000,"Pulp_Fiction,AmerricanHistoryX,TheWolfofWallSt...",BlacKkKlansman
2,2,0.850000,0.593023,0.698630,86.000000,"Pulp_Fiction,AmerricanHistoryX,TheWolfofWallSt...",BlacKkKlansman
3,accuracy,0.940426,0.940426,0.940426,0.940426,"Pulp_Fiction,AmerricanHistoryX,TheWolfofWallSt...",BlacKkKlansman
4,macro avg,0.817137,0.731265,0.766491,1645.000000,"Pulp_Fiction,AmerricanHistoryX,TheWolfofWallSt...",BlacKkKlansman


In [27]:
normal_dict = get_precision_recall_f1('0', df_cv)
offensive_dict = get_precision_recall_f1('1',df_cv)
hate_dict = get_precision_recall_f1('2',df_cv)

#### Aggregated classification results for all 6 folds

In [28]:
df_result = pd.DataFrame([normal_dict, offensive_dict, hate_dict])
df_result

,label,precision,recall,f1
0,0,0.966308,0.980306,0.973222
1,1,0.797687,0.755039,0.775327
2,2,0.787202,0.685645,0.678414


In [29]:
for cr in classification_reports:
  print(cr)

{'0': {'precision': 0.9631120053655265, 'recall': 0.9822161422708618, 'f1-score': 0.9725702675245513, 'support': 1462}, '1': {'precision': 0.6382978723404256, 'recall': 0.6185567010309279, 'f1-score': 0.6282722513089005, 'support': 97}, '2': {'precision': 0.85, 'recall': 0.5930232558139535, 'f1-score': 0.6986301369863014, 'support': 86}, 'accuracy': 0.9404255319148936, 'macro avg': {'precision': 0.8171366259019841, 'recall': 0.7312653663719145, 'f1-score': 0.7664908852732512, 'support': 1645}, 'weighted avg': {'precision': 0.9380453771801952, 'recall': 0.9404255319148936, 'f1-score': 0.9379467059444859, 'support': 1645}}
{'0': {'precision': 0.9706103993971364, 'recall': 0.9662415603900976, 'f1-score': 0.968421052631579, 'support': 1333}, '1': {'precision': 0.8302583025830258, 'recall': 0.8490566037735849, 'f1-score': 0.8395522388059701, 'support': 265}, '2': {'precision': 0.8333333333333334, 'recall': 0.8333333333333334, 'f1-score': 0.8333333333333334, 'support': 24}, 'accuracy': 0.945